#### Inspiration
I was inspired by **Zalgo text** from the Week 1 readings. I'd seen this type of "cursed" text before but didn't know what it was called or how it was made. The idea of abusing Unicode diacritics is interesting and I wanted to see how else I could use it. The readings also inspired me to work with **emoji**, I wanted to use those in Zalgo text somehow.

I quickly realized that emoji can't be stacked in Zalgo text because of the specific Unicode characters that are being abused. Diacritics are the only things that can stack like that. I pivoted my idea to using emoji characters as the base characters that receive Zalgo text diacritics.

Since the diacritics can only vertically stack on the base characters, I decided to work with imagery of flowers, vines, or grass since those grow up or down from a base. I wanted to work that imagery into a text about the same topic.

In [102]:
import unicodedata
import random

In [20]:
!curl -L -O https://www.unicode.org/Public/13.0.0/ucd/UnicodeData.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1808k  100 1808k    0     0  2656k      0 --:--:-- --:--:-- --:--:-- 2667k


In [21]:
unicode = []
for line in open("UnicodeData.txt"):
    line = line.strip()
    fields = line.split(';')
    row = {
        'char': chr(int(fields[0], 16)),
        'name': fields[1],
        'category': fields[2]
    }
    unicode.append(row)

I wanted to try searching Unicode characters programmatically to find some interesting flower characters to work with. This was harder than I thought because many flower-related characters don't have an available glyph on my computer. I also didn't want the first 2 results shown below that aren't emojis.

In [63]:
flowerTerms = ["FLOWER", "ROSE", "BLOSSOM"]
hasFlowerName = lambda name: any(flowerTerm in name for flowerTerm in flowerTerms)
flowers = [(row['char'], row['name']) for row in unicode if hasFlowerName(row['name'])]
flowers

[('⁕', 'FLOWER PUNCTUATION MARK'),
 ('⚘', 'FLOWER'),
 ('⯘', 'PROSERPINA'),
 ('𐇵', 'PHAISTOS DISC SIGN ROSETTE'),
 ('🌸', 'CHERRY BLOSSOM'),
 ('🌹', 'ROSE'),
 ('🌻', 'SUNFLOWER'),
 ('🌼', 'BLOSSOM'),
 ('🎕', 'BOUQUET OF FLOWERS'),
 ('🎴', 'FLOWER PLAYING CARDS'),
 ('🏵', 'ROSETTE'),
 ('🏶', 'BLACK ROSETTE'),
 ('💮', 'WHITE FLOWER'),
 ('🥀', 'WILTED FLOWER')]

Instead of using this list I handpicked (pun intended) a few flower emojis that are mentioned in the *Alice in Wonderland* chapter below. I ran into a lot of problems with this still. It seems to be a limitation of Unicode that diacritics cannot be placed on emoji characters. I'll demonstrate the problem below when I define the Zalgo text function.

I again used the Unicode dataset to find diacritics for my Zalgo text generator. I ended up only wanting diacritics that stack above a base character, so I included that in my filter criteria. Again, many of the characters returned don't have glyphs on my computer. I limited the results to 9 diacritics that do render correctly.

In [257]:
diacritics = [row['char'] for row in unicode if 'ABOVE' in row['name'] and 'COMBINING' in row['name']]
diacritics = diacritics[:9]
diacritics

['̇', '̉', '̊', '̍', '̎', '̒', '̓', '̔', '̕']

Since I mentioned that emoji don't work with diacritics, I came up with a new plan for how to make my Zalgo flowers visually stand out against the text. I've used ANSI escape codes before to style Python print output for a commandline card game. These escape codes are special strings that when printed can change the text and background color of other characters. I used these color codes to style the text used for my Zalgo flowers.

In [344]:
green = lambda x: "\033[1;32;42m" + x + "\033[0m"
yellow = lambda x: "\033[1;33;43m" + x + "\033[0m"
red = lambda x: "\033[1;31;41m" + x + "\033[0m"
pink = lambda x: "\033[1;35;45m" + x + "\033[0m"
blue = lambda x: "\033[1;34;44m" + x + "\033[0m"

print(red("roses are red"))
print(blue("violets are blue"))
print(green("leaves are green"))
print(yellow("daisies are yellow"))
print(pink("lilies are pink"))

roses are red
violets are blue
leaves are green
daisies are yellow
lilies are pink


I researched more detail on how the ANSI color codes work:
- \033 is an octal number, representing an ASCII escape character (decimal number 27)
- [ is the start of a control sequence
- 1 sets text to bold/bright mode
- ; separates parameters within the control sequence
- 32 sets foreground text to green. 42 is background color green
- m is the end of a control sequence

After setting a color code you have to reset it back to normal so the following characters are styled normally.


In [332]:
def zalgo(word, color):
    z = []
    for letter in word:
        z.append(color(letter))
        for i in range(random.randint(10, 100)):
            z.append(color(random.choice(diacritics)))
    return ''.join(z)

In [341]:
print('\n'*5)
print(zalgo("aaa", red))







a̎̒̊̒̔̉̎̓̍̍̎̎̉̎̓̒̔̔̒̇̒̒̊̉̔̊̓̔̇̍̔̊̒̍̔̍̊̎̉̒̒̓̒̒̊̍̓̉̕̕̕̕̕̕a̎̓̓̔̒̇̊̊̊̒̔̒̍̔̎̊̇̇̍̒̎̔̇̒̇̓̊̉̍̇̇̊̍̉̍̊̎̉̊̊̉̓̎̔̇̉̓̉̒̍̍̓̔̔̕̕̕̕̕̕a̒̊̍̒̓̒̒̉̍̍̎̊̇̍̔̊̓̍̒̉̒̍̊̒̔̎̇̎̍̊̔̊̓̇̉̒̊̔̎̇̔̍̒̍̊̔̇̎̊̊̉̇̎̓̉̊̎̓̓̍̒̒̉̓̊̓̎̉̎̓̍̉̒̎̉̕̕̕̕̕̕̕


I used these color lambda functions in conjunction with my `zalgo` function to color each base character and diacritic. I encountered a few more limitations here:
- Diacritics won't maintain a background color if they stack too high
- Diacritics can't be a different foreground color than the base character

I really wanted the base characters to be a flower color and the Zalgo text to be green to represent leaves, vines, or grass. But that seems to be impossible.

Here's an example of trying to Zalgo-ify an emoji character. The diacritics all go to the side or disappear.

In [330]:
print('\n'*5)
print(zalgo("🌸🌸🌸", pink))







🌸̍̓̓̊̇̉̔̍̇̉̉̇̍̉̊̇̍̎̎̉̒̍̕̕̕̕̕🌸̓̊̔̍̇̔̎̎̎̇̒̊̍̉̊̎̒̒̓̒̔̔̒̒̔̓̉̍̉̉̊̎̊̇̎̉̎̇̇̒̒̔̓̎̔̔̉̒̔̉̊̇̉̔̊̇̒̍̉̍̇̒̊̉̓̉̊̍̍̔̍̔̓̊̓̊̊̉̇̔̎̇̔̍̊̉̇̓̓̔̔̓̒̒̓̊̍̉̍̇̓̓̒̊̓̇̔̉̔̔̉̎̍̎̒̔̍̊̊̍̓̔̇̇̒̓̎̓̓̊̍̒̇̔̒̒̎̎̇̕̕̕̕̕̕̕̕̕̕̕̕̕̕🌸̒̓̇̎̓̎̊̎̊̉̇̇̍̎̓̉̔̇̍̊̍̒̉̇̉̍̇̒̓̇̊̇̒̒̔̇̉̎̒̎̒̍̔̔̉̊̒̒̒̊̒̒̊̒̉̓̉̍̉̊̔̍̓̓̍̇̔̍̎̎̊̒̔̒̓̔̎̒̇̕̕̕̕̕̕̕̕̕̕̕̕


Time to work with the actual text document. I chose some words to associate with certain colors. It's almost Valentines Day so I was inspired by the cheesy "Roses are red, violets are blue..." poems.

In [286]:
colorToWords = {
    "yellow": ["daisy", "daisies", "Daisy"],
    "red": ["rose", "roses", "Rose"],
    "pink": ["lily", "Lily"],
    "blue": ["violet", "Violet", "dahlia"],
}

In [334]:
poem = open("alice.txt").read()

for color, words in colorToWords.items():
    for word in words:
        z = ""
        if color == "red":
            z = zalgo(word, red)
        elif color == "pink":
            z = zalgo(word, pink)
        elif color == "yellow":
            z = zalgo(word, yellow)
        elif color == "blue":
            z = zalgo(word, blue)
            
        poem = poem.replace(word, z)

In [335]:
print(poem)

Chapter II: The Garden of Live Flowers

‘I should see the garden far better,’ said Alice to herself, ‘if I could get to the top of that hill: and here’s a path that leads straight to it—at least, no, it doesn’t do that—’ (after going a few yards along the path, and turning several sharp corners), ‘but I suppose it will at last. But how curiously it twists! It’s more like a corkscrew than a path! Well, this turn goes to the hill, I suppose—no, it doesn’t! This goes straight back to the house! Well then, I’ll try it the other way.’

And so she did: wandering up and down, and trying turn after turn, but always coming back to the house, do what she would. Indeed, once, when she turned a corner rather more quickly than usual, she ran against it before she could stop herself.

‘It’s no use talking about it,’ Alice said, looking up at the house and pretending it was arguing with her. ‘I’m not going in again yet. I know I should have to get through the Looking-glass again—back into the old roo

#### Results
Overall I'm happy with how this turned out, even without using emoji. I like the aesthetic of blobs of color crawling among the lines of text. It may look more like some alien mold than flowers... but I like how the underlying text is still readable.

It's also an interesting visualization of the characters in this chapter. The splashes of yellow make it easy to see that the daisies are involved in the beginning of the chapter, and the roses are more involved towards the end.